In [23]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
#import warnings; warnings.simplefilter('ignore')
import time
import imblearn
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp
from sklearn.metrics import roc_curve, auc
from sklearn.decomposition import NMF
from sklearn import cluster
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, train_test_split
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.combine import SMOTETomek
from sklearn import neighbors
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold #for K-fold cross
from sklearn.model_selection import cross_val_score #scoreevaluation
from sklearn.model_selection import cross_val_predict #prediction
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.model_selection import train_test_split
from imblearn.datasets import make_imbalance
from imblearn.under_sampling import NearMiss # underSampling
from imblearn.over_sampling import SMOTE # OverSampling
from imblearn.combine import SMOTEENN # Combination of the 2
from imblearn.pipeline import make_pipeline
from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn import svm, tree, neighbors
from sklearn.naive_bayes import GaussianNB, MultinomialNB,BernoulliNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.decomposition import LatentDirichletAllocation as LDA
from tqdm import tqdm_notebook as tqdm

In [10]:
df=pd.read_csv ('dataset.csv')
df.tail(12)

,Unnamed: 0,label,text,databersih
4536,387,0.0,Klarifikasi Terkait Perbedaan Hasil Situng KPU...,klarifikasi kait beda hasil situng kpu pindah ...
4537,388,0.0,“PKS dan Gerindra Purwakarta Bantah Pasang Spa...,pks gerindra purwakarta bantah pasang spanduk ...
4538,389,0.0,Universitas Muhammadiyah Kupang Bantah Akan Me...,universitas muhammadiyah kupang bantah mengini...
4539,390,0.0,Klarifikasi Atas Isu OTT Bupati Kapuas Oleh KPK,klarifikasi isu ott bupati kapuas kpk
4540,391,0.0,“Kronologis Polisi Berondong Sedan Terobos Raz...,kronologis polisi berondong sedan terobos razi...
4541,392,1.0,Foto Sekarang malesiya sapu habis penduduk asi...,malesiya sapu habis duduk asing perhati perhat...
4542,393,1.0,Foto Sekarang malesiya sapu habis penduduk asi...,malesiya sapu habis duduk asing perhati perhat...
4543,394,0.0,Klarifikasi Polres Bogor Terkait Pembuatan SIM...,klarifikasi polres bogor kait sim tes 150 ac...
4544,395,0.0,Ketua INASGOC Bantah Indonesia Bermain Curang ...,ketua inasgoc bantah indonesia main curang cab...
4545,396,0.0,Klarifikasi “Kenapa sholat di tempat ibadah ag...,klarifikasi sholat ibadah agama darurat


In [4]:
# Bentuk VSM-nya
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(lowercase=True, smooth_idf= True, sublinear_tf=True, 
                                   ngram_range=(1, 2), max_df=0.90, min_df=2)

In [12]:
##TFIDF
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import gensim
#vectorizer = TfidfVectorizer(ngram_range=(1, 2))
vectorizer = TfidfVectorizer(max_df=0.9, min_df=5, ngram_range=(1, 2))

listdata=df['databersih'].values.astype('object')
listdata = [d for d in listdata]

listdata
v = TfidfVectorizer(decode_error='replace', encoding='utf-8')
tfidf = v.fit_transform(df['databersih'].values.astype('U'))
y = df.iloc[:, 2].values
print(tfidf.shape, len(y)) 

(4548, 13786) 4548


max_df= 0.9 dan min_df=5 , nilai max_df tersebut artinya kita mengabaikan kata yang muncul lebih dari 90% dokumen dan nilai min_df artinya kita mengabaikan kata yang muncul kurang dari 5 dokumen, tf idf yang didapatkan adalah (4548, 13786) 4548 kalimat dengan 13786 kata unik

In [24]:
from sklearn.model_selection import train_test_split
seed = 1
X = df['databersih'].values.astype('U')
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=seed)
X_train = vectorizer.fit_transform(X_train) # "Fit_Transform"
X_test = vectorizer.transform(X_test) # Perhatikan disini hanya "Transform"

#print(X_train.shape, X_test.shape)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(3183, 2310)
(1365, 2310)
(3183,)
(1365,)


# SVM

In [14]:
#svm
from sklearn import svm
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

dSVM = svm.SVC( kernel = 'linear')
dSVM.fit(X_train, y_train)
y_SVM = dSVM.predict(X_test)
print('Akurasi = ', accuracy_score(y_test, y_SVM))
print(confusion_matrix(y_test, y_SVM))
print(classification_report(y_test, y_SVM))

Akurasi =  0.8227106227106227
[[126 212]
 [ 30 997]]
              precision    recall  f1-score   support

         0.0       0.81      0.37      0.51       338
         1.0       0.82      0.97      0.89      1027

    accuracy                           0.82      1365
   macro avg       0.82      0.67      0.70      1365
weighted avg       0.82      0.82      0.80      1365



In [29]:
#under sampling
rm = RandomUnderSampler(random_state=1)
X_rm, y_rm = rm.fit_resample(X_train, y_train)
dSVM = svm.SVC( kernel = 'linear')
dSVM.fit(X_rm, y_rm)
y_dSVM = dSVM.predict(X_test); del dSVM
print('UnderSamplingResults:\n',classification_report_imbalanced(y_test, y_dSVM))
print('Akurasi undersampling test= ', accuracy_score(y_test,y_dSVM))
#print(classification_report(y_test, y_dSVM))

UnderSamplingResults:
                    pre       rec       spe        f1       geo       iba       sup

        0.0       0.50      0.65      0.78      0.56      0.71      0.50       338
        1.0       0.87      0.78      0.65      0.83      0.71      0.52      1027

avg / total       0.78      0.75      0.68      0.76      0.71      0.51      1365

Akurasi undersampling test=  0.7516483516483516


In [30]:
#oversampling
ros = RandomOverSampler(random_state=1)
X_ros, y_ros = ros.fit_resample(X_train, y_train)
dSVM = svm.SVC( kernel = 'linear')
dSVM.fit(X_ros, y_ros)
y_dSVM = dSVM.predict(X_test); del dSVM
print('OverSamplingResults:\n',classification_report_imbalanced(y_test, y_dSVM))
print('Akurasi oversampling test = ', accuracy_score(y_test,y_dSVM))


OverSamplingResults:
                    pre       rec       spe        f1       geo       iba       sup

        0.0       0.55      0.58      0.84      0.56      0.70      0.47       338
        1.0       0.86      0.84      0.58      0.85      0.70      0.50      1027

avg / total       0.78      0.78      0.64      0.78      0.70      0.49      1365

Akurasi oversampling test =  0.7772893772893773


In [37]:
#both
#smt = SMOTEENN (ratio='auto')
smt = SMOTEENN(random_state=1)
X_smt, y_smt = smt.fit_resample(X_train, y_train)
dSVM = svm.SVC( kernel = 'linear')
dSVM.fit(X_rm, y_rm)
y_dSVM = dSVM.predict(X_test); del dSVM
print('CombinationResults:\n',classification_report_imbalanced(y_test, y_dSVM))
print('Akurasi combination = ', accuracy_score(y_test, y_dSVM))



CombinationResults:
                    pre       rec       spe        f1       geo       iba       sup

        0.0       0.50      0.65      0.78      0.56      0.71      0.50       338
        1.0       0.87      0.78      0.65      0.83      0.71      0.52      1027

avg / total       0.78      0.75      0.68      0.76      0.71      0.51      1365

Akurasi combination =  0.7516483516483516


In [39]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE

smot = SMOTE(sampling_strategy='all')
X_smot, y_smot = smot.fit_resample(X_train, y_train)
dSVM = svm.SVC( kernel = 'linear')
dSVM.fit(X_smot, y_smot)
y_dSVM = dSVM.predict(X_test); del dSVM
print('CombinationResults:\n',classification_report_imbalanced(y_test, y_dSVM))
print('Akurasi combination = ', accuracy_score(y_test, y_dSVM))

CombinationResults:
                    pre       rec       spe        f1       geo       iba       sup

        0.0       0.50      0.63      0.79      0.55      0.70      0.49       338
        1.0       0.87      0.79      0.63      0.83      0.70      0.50      1027

avg / total       0.77      0.75      0.67      0.76      0.70      0.50      1365

Akurasi combination =  0.7501831501831502


# Reglog

In [15]:
#logistic regression
from sklearn.linear_model import LogisticRegression
model_logreg = LogisticRegression(solver="lbfgs", multi_class="auto")
model_logreg.fit(X_train, y_train)
Y_pred = model_logreg.predict(X_test)
print("Accuracy:", accuracy_score(Y_pred, y_test))
print(confusion_matrix(y_test, Y_pred))
print(classification_report(y_test, Y_pred))

Accuracy: 0.8212454212454212
[[ 105  233]
 [  11 1016]]
              precision    recall  f1-score   support

         0.0       0.91      0.31      0.46       338
         1.0       0.81      0.99      0.89      1027

    accuracy                           0.82      1365
   macro avg       0.86      0.65      0.68      1365
weighted avg       0.84      0.82      0.79      1365



In [40]:
#under sampling
rm = RandomUnderSampler(random_state=1)
X_rm, y_rm = rm.fit_resample(X_train, y_train)
model_logreg = LogisticRegression(solver="lbfgs", multi_class="auto")
model_logreg.fit(X_rm, y_rm)
y_model_logreg = model_logreg.predict(X_test); del model_logreg
print('UnderSamplingResults:\n',classification_report_imbalanced(y_test, y_model_logreg))
print('Akurasi undersampling test= ', accuracy_score(y_test,y_model_logreg))


UnderSamplingResults:
                    pre       rec       spe        f1       geo       iba       sup

        0.0       0.53      0.64      0.81      0.58      0.73      0.52       338
        1.0       0.87      0.81      0.64      0.84      0.73      0.53      1027

avg / total       0.79      0.77      0.69      0.78      0.73      0.53      1365

Akurasi undersampling test=  0.7728937728937729


In [17]:
#random forest
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_rf = rf.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_rf))
print(confusion_matrix(y_test, y_rf))
print(classification_report(y_test, y_rf))

Accuracy: 0.8175824175824176
[[155 183]
 [ 66 961]]
              precision    recall  f1-score   support

         0.0       0.70      0.46      0.55       338
         1.0       0.84      0.94      0.89      1027

    accuracy                           0.82      1365
   macro avg       0.77      0.70      0.72      1365
weighted avg       0.81      0.82      0.80      1365



# Under sampling